# Simulate workers at a call center processing calls

Current constraints:
1. One shift of workers
2. Every call takes the same amount of time
3. Workers do not take breaks
4. Callers never hang up

Free parameters:
1. Number of workers

Goal:
- Determine which calls were processed successfully and which were missed

In [1]:
from queue import Queue
from queue import LifoQueue
from queue import PriorityQueue
from collections import deque

from datetime import datetime
from datetime import date
from datetime import time
from datetime import timedelta

## Read all individual call events

In [2]:
def read_calls(fname):
    calls = []
    with open(fname, "r") as fin:
        for line in fin:
            line = line.rstrip("\n")
            
            # use time.fromisoformat() in future (python 3.7+)
            hour, minute, second = map(int, line.split(":"))
            calls.append(time(hour=hour, minute=minute, second=second))
            
    return calls

### Generate call stack

In [3]:
def get_call_stack(fname):
    calls = read_calls(fname)
    
    call_stack = LifoQueue()
    for call in reversed(calls):
        call_stack.put(call)
        
    return call_stack

## Main loop of time

so priority queue also sorts by secondary item in addition to the priority
times are sorted in increasing time

In [4]:
# define constants

TIME_RESOLUTION = timedelta(seconds=1)

# how long every caller spends talking to an agent
# becomes a free parameter later on
TALK_TIME = timedelta(minutes=10)

# when the workers start and stop working
START_TIME = time(hour=5, minute=0)
END_TIME = time(hour=22, minute=30)

#--------------------------------------------------------------------------

# define free parameters (later inputs to our function)

num_workers = 10

#--------------------------------------------------------------------------


cur_time = datetime.combine(date.min, START_TIME)
stop_time = datetime.combine(date.min, END_TIME)


call_times_fname = "data/call_times_uniform.txt"
call_stack = get_call_stack(call_times_fname)

hold_line = Queue() # callers who are on hold because all agents are busy

# list of agents who are not currently busy with callers
free_workers = deque(i for i in range(num_workers))

# priority queue of when the calls finish with the worker number
work_line = PriorityQueue()

successful_calls = 0

while cur_time < stop_time:

    # add new callers to wait queue
    while not call_stack.empty():
        new_call = call_stack.get()

        new_call_datetime = datetime.combine(date.min, new_call)

        if cur_time == new_call_datetime:
            hold_line.put(new_call)
        else:
            assert cur_time < new_call_datetime

            call_stack.put(new_call)
            break


    # free up any workers who are done
    while not work_line.empty():
        earliest, worker_id = work_line.get()

        if cur_time < earliest:
            work_line.put((earliest, worker_id))
            break
        else:
            assert cur_time == earliest

            successful_calls += 1
            free_workers.append(worker_id)


    # assign free workers to new callers
    while free_workers and not hold_line.empty():
        worker_id = free_workers.popleft()
        caller_time = hold_line.get()

        # check to see if caller should hang up in future
        finish_datetime = cur_time + TALK_TIME

        work_line.put((finish_datetime, worker_id))


    cur_time += TIME_RESOLUTION    

In [5]:
successful_calls

1015